# Pré-Processamento - Leitura e Limpeza

## Resumo do alcance dessa etapa

Os dados são importados de diferentes fontes e unidos em uma única base.
A cada variável é dado um nome informativo, a base é limpada de inconsitências e ausências e, por fim, a desigualdade (medida como um índice Gini) é classificada em duas categorias: alto e baixo.

Fontes dados:

[Censo Agro 2017 Mapa Interativo - IBGE](https://mapasinterativos.ibge.gov.br/agrocompara/)

[Datasus - Min Saúde](http://tabnet.datasus.gov.br/cgi/ibge/censo/cnv/ginibr.def)

O mapa Interativo não disponibiliza o link para download direto. Subimos o [arquivo](https://github.com/feliciov/agro_online/raw/main/nbs/data_gini/mun_agro.zip) (baixado no dia XX/XX/XXXX) em nosso repositório.

## Detalhamento da etapa

### Imports

In [1]:
import pandas as pd
import numpy as np

# carregamento de dados armazenado em shapefile
import geopandas as gpd

### Leitura e Limpesa das bases de dados

In [2]:
df = gpd.read_file("https://github.com/feliciov/agro_online/raw/main/nbs/data_gini/mun_agro.zip")

In [3]:
y = pd.read_csv(
    "http://tabnet.datasus.gov.br/cgi/ibge/censo/bases/ginibr.csv",
    encoding='latin1',
    decimal=',',
    sep=";",
    skiprows=2,
    skipfooter=2,
    engine="python",
)

In [4]:
df.head()

,OBJECTID,MUNICIPIO,Shape_Leng,Shape_Area,V1,V2,V3,V4,V5,V6,...,GEO,V32,V33,V34,V35,V36,V37,V38,V39,geometry
0,1,Lupionópolis - PR,0.514701,0.010596,173.0,59.7001,3.23121,165.7380,852.3680,124.3040,...,4113809.0,2.457880,1.156070,48.55490,8.586240,8.09249,68.78610,71.0983,36.9942,"POLYGON Z ((-51.63021 -22.66372 0.00000, -51.6..."
1,2,Águas de Lindóia - SP,0.344094,0.004856,69.0,28.4016,3.42029,36.3902,80.0584,133.4310,...,3500501.0,2.185360,5.797100,39.13040,2.435280,13.04350,49.27540,60.8696,46.3768,"POLYGON Z ((-46.61019 -22.43936 0.00000, -46.6..."
2,3,Uruará - PA,5.505977,0.882292,3993.0,137.3820,3.05109,1556.9600,4003.6200,800.7230,...,1508159.0,0.947595,0.075132,30.12770,3.377250,2.17881,5.76008,84.0471,64.1122,"POLYGON Z ((-53.85954 -2.88750 0.00000, -53.85..."
3,4,São Leopoldo - RS,0.505968,0.009694,65.0,26.7850,3.44615,85.3800,85.3800,18.9733,...,4318705.0,1.906410,1.538460,9.23077,0.754856,20.00000,26.15380,53.8462,40.0000,"POLYGON Z ((-51.17861 -29.69309 0.00000, -51.1..."
4,5,Vargem - SP,0.535624,0.012407,228.0,31.9352,3.11404,52.0186,127.1570,54.4957,...,3556354.0,1.693530,0.877193,12.71930,2.231690,14.03510,55.26320,54.8246,51.7544,"POLYGON Z ((-46.39678 -22.81069 0.00000, -46.3..."


In [5]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 45 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   OBJECTID    5570 non-null   int64   
 1   MUNICIPIO   5570 non-null   object  
 2   Shape_Leng  5570 non-null   float64 
 3   Shape_Area  5570 non-null   float64 
 4   V1          5570 non-null   float64 
 5   V2          5570 non-null   float64 
 6   V3          5570 non-null   float64 
 7   V4          5570 non-null   float64 
 8   V5          5570 non-null   float64 
 9   V6          5570 non-null   float64 
 10  V7          5570 non-null   float64 
 11  V8          5570 non-null   float64 
 12  V9          5570 non-null   float64 
 13  V10         5570 non-null   float64 
 14  V11         5570 non-null   float64 
 15  V12         5570 non-null   float64 
 16  V13         5570 non-null   float64 
 17  V14         5570 non-null   float64 
 18  V15         5570 non-null   float64 
 19

O arquivo utilizado veio com as variáveis nomeadas de maneira pouco informativa.

Com auxílio do [dicionário de dados](https://mapasinterativos.ibge.gov.br/agrocompara/dados/def_indicadores.pdf), mudamos os rótulos para que sejam mais informativos.

In [6]:
dict_rename = {
    'V1' :'Estabelecimento Agropecuário (N)',
    'V2' :'Área média (ha)',
    'V3' : 'Pessoal Ocupado / Estabelecimento (Pessoa)',
    'V4' :'Área lavoura/ Adubadeira (ha)',
    'V5' :'Área lavoura/ Colheitadeira(ha)',
    'V6' :'Área lavoura/ Semeadeira(ha)',
    'V7' :'Área lavoura/ Trator(ha)',
    'V8' :'Atividade-Lavoura Temporária (%)',
    'V9' :'Atividade-Lavoura Permanente(%)',
    'V10' :'Atividade-Pecuária(%)',
    'V11' :'Atividade-Horticultura&Floricultura(%)',
    'V12' :'Atividade-Sementes&Mudas(%)',
    'V13' :'Atividade-Produção Florestal(%)',
    'V14' :'Atividade-Pesca(%)',
    'V15' :'Atividade-Aquicultura(%)',
    'V16' :'Uso das terras-Lavoura(%)',
    'V17' :'Uso das terras-Pastagem (%)',
    'V18' :'Aves-Corte (%)',
    'V19' :'Aves-Ovos(%)',
    'V20' :'Bovinos-Corte (%)',
    'V21' :'Bovinos-Leite(%)',
    'V22' :'Rendimento-Arroz (kg/ha)',
    'V23' :'Rendimento-Cana (kg/ha)',
    'V24' :'Rendimento-Mandioca (kg/ha)',
    'V25' :'Rendimento-Milho (kg/ha)',
    'V26' :'Rendimento-Soja (kg/ha)',
    'V27' :'Rendimento-Trigo(kg/ha)',
    'V28' :'Rendimento-Cacau(kg/ha)',
    'V29' :'Rendimento-Café(kg/ha)',
    'V30':'Rendimento-Laranja(kg/ha)',
    'V31':'Rendimento-Uva(kg/ha)',
    'V32':'Carga de Bovinos (n/ha)',
    'V33':'Cisterna (%)',
    'V34':'Utilização de Agrotóxicos (%)',
    'V35':'Despesa com Agrotóxicos (%)',
    'V36':'Uso de irrigação (%)',
    'V37':'Assistência Técnica (%)',
    'V38':'Agricultura familiar (%)',
    'V39':'Produtor com escolaridade até Ensino Fundamental(%)',
}

In [7]:
df = df.rename(columns=dict_rename)

Nem todas as variáveis são informativas para essa análise (como as coordenadas dos shapes, por exemplo).
Abaixo, selecionamos as que não foram julgadas relevantes.

In [8]:
variaveis_relevantes = [
    'GEO', 'MUNICIPIO', 'Estabelecimento Agropecuário (N)',
    'Área média (ha)', 'Pessoal Ocupado / Estabelecimento (Pessoa)',
    'Área lavoura/ Adubadeira (ha)', 'Área lavoura/ Colheitadeira(ha)',
    'Área lavoura/ Semeadeira(ha)', 'Área lavoura/ Trator(ha)',
    'Atividade-Lavoura Temporária (%)', 'Atividade-Lavoura Permanente(%)',
    'Atividade-Pecuária(%)', 'Atividade-Horticultura&Floricultura(%)',
    'Atividade-Sementes&Mudas(%)', 'Atividade-Produção Florestal(%)',
    'Atividade-Pesca(%)', 'Atividade-Aquicultura(%)',
    'Uso das terras-Lavoura(%)', 'Uso das terras-Pastagem (%)',
    'Aves-Corte (%)', 'Aves-Ovos(%)', 'Bovinos-Corte (%)',
    'Bovinos-Leite(%)', 'Rendimento-Arroz (kg/ha)',
    'Rendimento-Cana (kg/ha)', 'Rendimento-Mandioca (kg/ha)',
    'Rendimento-Milho (kg/ha)', 'Rendimento-Soja (kg/ha)',
    'Rendimento-Trigo(kg/ha)', 'Rendimento-Cacau(kg/ha)',
    'Rendimento-Café(kg/ha)', 'Rendimento-Laranja(kg/ha)',
    'Rendimento-Uva(kg/ha)', 'Carga de Bovinos (n/ha)',
    'Cisterna (%)', 'Utilização de Agrotóxicos (%)',
    'Despesa com Agrotóxicos (%)', 'Uso de irrigação (%)',
    'Assistência Técnica (%)', 'Agricultura familiar (%)',
    'Produtor com escolaridade até Ensino Fundamental(%)',
    ]

In [9]:
df = df[variaveis_relevantes]

A base do Datasus utilizou o código IBGE sem o dígito verificador. Criando coluna com essa informação para usar de chave na união entre as bases.

In [10]:
df['COD_MUN'] = df.GEO.astype(int).astype(str).str[:-1]

Com os dados do censo agropecuário melhor formatados, seguimos para processamento dos dados da FONTE

Primeira etapa é separar o código do município do nome.

In [11]:
y[['COD_MUN', 'NOME_MUN']] = y['Município'].str.split(' ', n=1, expand=True)

In [12]:
print(y.shape)
y.head()

(5565, 6)


,Município,1991,2000,2010,COD_MUN,NOME_MUN
0,110001 Alta Floresta D'Oeste,0.5983,0.5868,0.5893,110001,Alta Floresta D'Oeste
1,110037 Alto Alegre dos Parecis,...,0.508,0.5491,110037,Alto Alegre dos Parecis
2,110040 Alto Paraíso,...,0.6256,0.5417,110040,Alto Paraíso
3,110034 Alvorada D'Oeste,0.569,0.6534,0.5355,110034,Alvorada D'Oeste
4,110002 Ariquemes,0.5827,0.5927,0.5496,110002,Ariquemes


In [13]:
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5565 entries, 0 to 5564
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Município  5565 non-null   object 
 1   1991       5565 non-null   object 
 2   2000       5565 non-null   object 
 3   2010       5565 non-null   float64
 4   COD_MUN    5565 non-null   object 
 5   NOME_MUN   5565 non-null   object 
dtypes: float64(1), object(5)
memory usage: 261.0+ KB


In [14]:
y = y[['COD_MUN', '2010']].rename(columns={
    '2010':'gini'
})

### Criação das categorias

Gini foi binarizado em 'gini_alto' e 'gini_baixo', critério de corte no 50 percentil

In [15]:
# valor de corte
y.gini.quantile(q=.5)

0.5025

In [16]:
y.gini = np.select(
    [y['gini'] <= y['gini'].quantile(q=.5)],
    ['gini_baixo'], default='gini_alto'
)

### Merge

In [17]:
df = df.merge(y, on='COD_MUN')

In [18]:
df.shape

(5565, 43)

In [19]:
df.dropna().shape #inefetivo, o merge foi do tipo 'inner_join', não haviam NAs resultantes

(5565, 43)

Em seguida, separamos os labels dos municípios e as variáveis de interesse.

In [20]:
df.head()

,GEO,MUNICIPIO,Estabelecimento Agropecuário (N),Área média (ha),Pessoal Ocupado / Estabelecimento (Pessoa),Área lavoura/ Adubadeira (ha),Área lavoura/ Colheitadeira(ha),Área lavoura/ Semeadeira(ha),Área lavoura/ Trator(ha),Atividade-Lavoura Temporária (%),...,Carga de Bovinos (n/ha),Cisterna (%),Utilização de Agrotóxicos (%),Despesa com Agrotóxicos (%),Uso de irrigação (%),Assistência Técnica (%),Agricultura familiar (%),Produtor com escolaridade até Ensino Fundamental(%),COD_MUN,gini
0,4113809.0,Lupionópolis - PR,173.0,59.7001,3.23121,165.7380,852.3680,124.3040,53.27300,25.43350,...,2.457880,1.156070,48.55490,8.586240,8.09249,68.78610,71.0983,36.9942,411380,gini_baixo
1,3500501.0,Águas de Lindóia - SP,69.0,28.4016,3.42029,36.3902,80.0584,133.4310,8.00584,8.69565,...,2.185360,5.797100,39.13040,2.435280,13.04350,49.27540,60.8696,46.3768,350050,gini_baixo
2,1508159.0,Uruará - PA,3993.0,137.3820,3.05109,1556.9600,4003.6200,800.7230,114.85800,10.09270,...,0.947595,0.075132,30.12770,3.377250,2.17881,5.76008,84.0471,64.1122,150815,gini_alto
3,4318705.0,São Leopoldo - RS,65.0,26.7850,3.44615,85.3800,85.3800,18.9733,4.61514,27.69230,...,1.906410,1.538460,9.23077,0.754856,20.00000,26.15380,53.8462,40.0000,431870,gini_alto
4,3556354.0,Vargem - SP,228.0,31.9352,3.11404,52.0186,127.1570,54.4957,8.05922,11.84210,...,1.693530,0.877193,12.71930,2.231690,14.03510,55.26320,54.8246,51.7544,355635,gini_baixo


In [21]:
municipios = df['MUNICIPIO']
X = df.drop(columns=[
    'COD_MUN', 'GEO', 'MUNICIPIO'
])

In [22]:
df[X.duplicated()]

,GEO,MUNICIPIO,Estabelecimento Agropecuário (N),Área média (ha),Pessoal Ocupado / Estabelecimento (Pessoa),Área lavoura/ Adubadeira (ha),Área lavoura/ Colheitadeira(ha),Área lavoura/ Semeadeira(ha),Área lavoura/ Trator(ha),Atividade-Lavoura Temporária (%),...,Carga de Bovinos (n/ha),Cisterna (%),Utilização de Agrotóxicos (%),Despesa com Agrotóxicos (%),Uso de irrigação (%),Assistência Técnica (%),Agricultura familiar (%),Produtor com escolaridade até Ensino Fundamental(%),COD_MUN,gini
1407,3510609.0,Carapicuíba - SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,351060,gini_baixo
2626,3303203.0,Nilópolis - RJ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,330320,gini_baixo
3227,3505708.0,Barueri - SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350570,gini_alto
3890,3500600.0,Águas de São Pedro - SP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,350060,gini_alto
5473,2919926.0,Madre de Deus - BA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,291992,gini_alto


Os 5 municípios não tem dados do censo agro e serão removidos

In [23]:
municipios= municipios[~X.duplicated()]
X = X[~X.duplicated()]

A próxima etapa é escalar e codificar as features conforme necessário.

In [24]:
# editar metadata
# tags ['remove-input']

municipios.to_pickle('data_gini/municipios.pkl.xz')
X.to_pickle('data_gini/X.pkl.xz')